# Nose poke analysis

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


/tmp/ipykernel_99626/2820620140.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/projects/nose_poke_identifier'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import utilities.helper
import sleap.process_pose

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
NOSE_POKE_DF = pd.read_pickle(r"./proc/rce_pilot_2_nose_pokes.pkl")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
MED_PC_WIDTH = 29.5
MED_PC_HEIGHT = 24
FRAME_RATE = 22
WINDOW_SIZE = 25
DISTANCE_THRESHOLD = 2

In [9]:
START_STOP_FRAME_DF = pd.read_excel("./rce_per_subject_start_stop_video_frame.xlsx").dropna(subset=["file_path"])

In [10]:
NOSE_POKE_DF.head()

,session_dir,tone_frames,box_1_port_entry_frames,video_name,session_path,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,...,box_bottom_right,nose_index,entry_length,box_1_port_entry_start,box_1_port_entry_end,sliced_locations,nose_coordinates,nose_to_reward_port_distance,entry_winner,contested
0,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[192, 248]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[192745, 249350]",...,"[59.91472868217054, 34.137834036568215]",2,56,192,248,"{'1.1': [[[47.88 11.6 ], [46.72 12.984], [48....","{'1.1': [[48.3, 13.7], [47.84, 13.695], [47.75...","{'1.1': 22.351771133170907, '1.2': 0.988053123...",1.2,False
1,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[389, 405]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[389747, 407142]",...,"[59.91472868217054, 34.137834036568215]",2,16,389,405,"{'1.1': [[[46.1 33.44], [43.97 34.94], [46.16...","{'1.1': [[46.16, 36.0], [46.12, 35.97], [46.16...","{'1.1': 0.8902350171923165, '1.2': 25.90464148...",1.1,False
2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[916, 929]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[917544, 931741]",...,"[59.91472868217054, 34.137834036568215]",2,13,916,929,"{'1.1': [[[61.34 18. ], [58.6 18.38], [60.25...","{'1.1': [[60.25, 19.8], [60.78, 19.5], [60.78,...","{'1.1': 21.949273716111037, '1.2': 1.066196743...",1.2,False
3,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[929, 948]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[932141, 950539]",...,"[59.91472868217054, 34.137834036568215]",2,19,929,948,"{'1.1': [[[59.7 15.125], [60.84 16.25], [61....","{'1.1': [[61.47, 16.03], [61.22, 15.4], [61.0,...","{'1.1': 25.589320821862895, '1.2': 0.630309449...",1.2,False
4,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[961, 983]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[963342, 986354]",...,"[59.91472868217054, 34.137834036568215]",2,22,961,983,"{'1.1': [[[58.62 12.96], [60.28 14.03], [59.75...","{'1.1': [[59.75, 13.06], [59.88, 14.26], [59.9...","{'1.1': 26.99243314043086, '1.2': 0.6916250894...",1.2,False


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [12]:
NOSE_POKE_PKL = "{}_nose_poke_analysis.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [13]:
NOSE_POKE_DF.columns

Index(['session_dir', 'tone_frames', 'box_1_port_entry_frames', 'video_name',
       'session_path', 'all_subjects', 'first_timestamp', 'last_timestamp',
       'tone_timestamps', 'box_1_port_entry_timestamps', 'box_top_left',
       'box_top_right', 'reward_port', 'box_bottom_left', 'box_bottom_right',
       'nose_index', 'entry_length', 'box_1_port_entry_start',
       'box_1_port_entry_end', 'sliced_locations', 'nose_coordinates',
       'nose_to_reward_port_distance', 'entry_winner', 'contested'],
      dtype='object')

In [14]:
NOSE_POKE_DF[NOSE_POKE_DF["contested"] == True]

,session_dir,tone_frames,box_1_port_entry_frames,video_name,session_path,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,...,box_bottom_right,nose_index,entry_length,box_1_port_entry_start,box_1_port_entry_end,sliced_locations,nose_coordinates,nose_to_reward_port_distance,entry_winner,contested
154,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[9718, 9719]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[9735938, 9738333]",...,"[59.91472868217054, 34.137834036568215]",2,1,9718,9719,"{'1.1': [[[40.5 34.56], [39.4 35.7], [43.94 3...","{'1.1': [[43.94, 36.3]], '1.2': [[44.66, 36.16]]}","{'1.1': 1.4196744786109914, '1.2': 0.694881360...",1.2,True
155,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[9720, 9868]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[9739343, 9887730]",...,"[59.91472868217054, 34.137834036568215]",2,148,9720,9868,"{'1.1': [[[40.5 34.94], [38.9 36.3], [44.44 3...","{'1.1': [[44.44, 36.4], [44.97, 36.7], [44.78,...","{'1.1': 1.0098364634611707, '1.2': 0.588110045...",1.2,True
156,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[9870, 9899]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[9888732, 9917332]",...,"[59.91472868217054, 34.137834036568215]",2,29,9870,9899,"{'1.1': [[[42.22 34.97], [40.6 37.1], [44.84 3...","{'1.1': [[44.84, 36.72], [44.75, 36.72], [44.6...","{'1.1': 0.9734128200563306, '1.2': 0.598293383...",1.2,True
165,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[10516, 10518]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[10536335, 10539137]",...,"[59.91472868217054, 34.137834036568215]",2,2,10516,10518,"{'1.1': [[[39.97 34.56], [38.9 35.6], [44.44 3...","{'1.1': [[44.44, 36.0], [43.9, 36.0]], '1.2': ...","{'1.1': 0.855168345366441, '1.2': 0.4728801936...",1.2,True
167,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[10640, 10723]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[10660132, 10744137]",...,"[59.91472868217054, 34.137834036568215]",2,83,10640,10723,"{'1.1': [[[42.28 34.6 ], [40.88 36.1 ], [44.8 ...","{'1.1': [[44.8, 36.44], [44.56, 36.5], [44.28,...","{'1.1': 0.7517168454547564, '1.2': 1.168323123...",1.1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[29662, 29665]",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.4]",835680,66439621,"[[1099250, 1299250], [3499256, 3699252], [5799...","[29718041, 29720041]",...,"[59.48163265306122, 35.3757225433526]",2,3,29662,29665,"{'1.1': [[[45.1 37. ], [41.84 37. ], [44.7 3...","{'1.1': [[44.7, 38.12], [44.7, 38.12], [44.7, ...","{'1.1': 0.8135105323027886, '1.4': 0.850665028...",1.1,True
3279,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[29668, 29669]",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.4]",835680,66439621,"[[1099250, 1

In [15]:
NOSE_POKE_DF["sliced_locations"].iloc[0]["1.1"].shape

(56, 6, 2)

In [16]:
NOSE_POKE_DF["entry_length"].iloc[0]

56